# 1.2 Convolution Neural Networks (ConvNet/CNN) Digit dataset 

In [75]:
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from torch import optim, nn
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [76]:
# reading training datasets
data_train = np.loadtxt('dataset/pendigits.tra', delimiter=',')
data_test = np.loadtxt('dataset/pendigits.tes', delimiter=',')

In [77]:
#spliting the features and labels 
X_train = data_train[:, :-1] / 100.0
y_train = data_train[:, -1] 
X_test = data_test[:, :-1] / 100.0
y_test = data_test[:, -1]

In [78]:
#spliting the training and validation datasets 
X_train, x_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [79]:
# convert from numpy to tensor data type
X_train, y_train, x_valid, y_valid, X_test, y_test = map(
    torch.tensor, (X_train.astype('float32'), y_train.astype('int64'), 
                   x_valid.astype('float32'), y_valid.astype('int64'),
                  X_test.astype('float32'), y_test.astype('int64'))
)

In [80]:
# Combining to Tensor Dataset
train_ds = TensorDataset(X_train, y_train)
valid_ds = TensorDataset(x_valid, y_valid)
print(X_train.shape)
print(x_valid.shape)
print(X_test.shape)

torch.Size([5995, 16])
torch.Size([1499, 16])
torch.Size([3498, 16])


## Optimizing the Model Parameters 
### * To train a model we need a loss function 


In [81]:
loss_fn = F.cross_entropy

## Create fit() and get_data()
### * Calculating the loss for both the training set and the validation set 
### * loss_batch function which computes the loss for  batch
### * Passing an opt in for the training set and use it to perform backprop

In [82]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

# fit runs the necessary operations to train model 
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    los = []
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        los.append(val_loss)

    return los

#get_data returns dataloaders for the training and validation sets 
def get_data(train_ds, valid_ds, batch_size):
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(valid_ds, batch_size=batch_size * 2),
    )

## nn. Sequential
### * Defining a custom layer from a given function 
### *  Lambda will create a layer that we can use when defining a network with Sequential.

In [83]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

# funciton to reshape input
def preprocess(x, y):
    return x.view(-1, 1, 4, 4), y

# class to preprocess input
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

# CNNModel 
### * Two  convolution Layers 
### * Each followed by ReLU
### * Performing Average pooling at the end 

In [84]:
epochs = 100  # number of iterations
lr = 0.01  # learning rate
batch_size = 128  # batch size

#preprocessing input 
train_dl, valid_dl = get_data(train_ds, valid_ds, batch_size)
train_dl = WrappedDataLoader(train_dl, preprocess)
valid_dl = WrappedDataLoader(valid_dl, preprocess)

#convolutional layer 
model = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(6, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
)

#printing out model
print(model)
# using an optimize 
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

#training the model 
los = fit(epochs, model, loss_fn, opt, train_dl, valid_dl)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(6, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): AdaptiveAvgPool2d(output_size=1)
  (5): Lambda()
)


In [85]:
# predicting on test set 
pred_y = model(X_train.view(-1,1,4,4)).detach().numpy()
pred_y = np.argmax(pred_y, axis = 1)

# testing the accuracy 
accur_y = y_train.numpy()
print("Accuracy on the test set : {:2.0f} %".format(np.mean(accur_y==pred_y)*100))

Accuracy on the test set : 88 %
